In [1]:
# Loading of Env Vars to enable parameterized command line usage
import os

GRAPHLET_SIZE = int(os.environ.get('GRAPHLET_SIZE', 3))
DATASET = os.environ.get('DATASET', "yeastInter_st.txt")
EXPERIMENT_OUT = os.environ.get('EXPERIMENT_OUT', "yeastInter_st")
METRIC_NAME = os.environ.get('METRIC_NAME', "degree")

In [2]:
from pmotifs.analysis_utilities.metric_consolidation import metrics

potential_metrics = metrics.keys()
assert METRIC_NAME in potential_metrics

ModuleNotFoundError: No module named 'pmotifs'

In [ ]:
from pmotifs.analysis_utilities.loading import Result
from pmotifs.config import config

r = Result.load_result(
    config.DATASET_DIRECTORY /  DATASET,
    config.EXPERIMENT_OUT / EXPERIMENT_OUT,
    GRAPHLET_SIZE,
)

g = r.pmotif_graph
df = r.positional_metric_df

In [ ]:
randomized_results = Result.load_randomized_results(g, GRAPHLET_SIZE, supress_tqdm=True)

In [ ]:
from typing import Union

from pmotifs.GraphletPositionalMetrics import GraphletPositionalMetrics


def get_positional_metric(result: Result) -> Union[int, float]:
    """A wrapper to consolidate metrics"""
    return metrics[METRIC_NAME](result)

In [ ]:
df[METRIC_NAME] = get_positional_metric(r)

In [ ]:
for r in randomized_results:
    r.positional_metric_df[METRIC_NAME] = get_positional_metric(r)

randomized_results = {
    r.pmotif_graph: r.positional_metric_df
    for r in randomized_results
}

# Analysis

In [ ]:
# Extract Distribution of Metric for one dataframe
def extract_metric_distribution(df, metric_name):
    return dict(df.groupby("graphlet_class").agg(list)[metric_name])

extract_metric_distribution(df, METRIC_NAME)

In [ ]:
from pmotifs.graphlet_representation import graphlet_class_to_name

import matplotlib.pyplot as plt

def plot_distirbution_per_class(df, metric_name):
    distributions = extract_metric_distribution(df, metric_name)
    fig, axes = plt.subplots(1, len(distributions), figsize=(5*len(distributions), 5))
    
    for i, graphlet_class in enumerate(distributions.keys()):
        ax = axes[i]
        ax.hist(distributions[graphlet_class])
        ax.set_title(graphlet_class_to_name(graphlet_class))
    # return fig

plot_distirbution_per_class(df, METRIC_NAME)

In [ ]:
graphlet_classes = list(df.groupby("graphlet_class").agg(lambda i: None).index)
graphlet_classes

In [ ]:
from scipy.stats import mannwhitneyu
from statistics import median
from tqdm import tqdm
import pandas as pd

original_distribution = extract_metric_distribution(df, METRIC_NAME)

data = {}
for graphlet_class in graphlet_classes:
    data[graphlet_class] = []
    for r_df in tqdm(randomized_results.values()):
        random_distribution = extract_metric_distribution(r_df, METRIC_NAME)

        mwu_result = mannwhitneyu(
            original_distribution[graphlet_class],
            random_distribution[graphlet_class],
        )

        data[graphlet_class].append({
           "u-statistic": mwu_result.statistic,
           "p-value": mwu_result.pvalue,
           "sample-size": len(random_distribution[graphlet_class]),
           "sample-median": median(random_distribution[graphlet_class]),
           "original-size": len(original_distribution[graphlet_class]),
           "original-median": median(original_distribution[graphlet_class]),
       })

result_dfs = {
    graphlet_class: pd.DataFrame(d)
    for graphlet_class, d in data.items()
}

In [ ]:
def echo_results(df, alpha_global=0.05):
    # Bonferroni Correction
    alpha_local = alpha_global / df.shape[0]

    rejected_rows = df[df["p-value"] < alpha_local]
    print(f"The original is not like the random in {rejected_rows.shape[0]} of {df.shape[0]} cases!")

In [ ]:
import matplotlib.pyplot as plt

def plot_results(df, graphlet_class):
    fig, (size_ax, median_ax) = plt.subplots(1,2, figsize=(10, 5))

    df[["sample-size"]].plot.hist(ax=size_ax)
    size_ax.axvline(result_df["original-size"][0], label="original", color="tab:orange")
    size_ax.set_title("sample-size")
    size_ax.legend()
    size_ax.set_xlabel(METRIC_NAME)

    df[["sample-median"]].plot.hist(ax=median_ax)
    median_ax.axvline(result_df["original-median"][0], label="original", color="tab:orange")
    median_ax.set_title("sample-median")
    median_ax.legend()
    median_ax.set_xlabel(METRIC_NAME)
    
    fig.suptitle(graphlet_class_to_name(graphlet_class))

In [ ]:
for graphlet_class, result_df in result_dfs.items():
    print(graphlet_class_to_name(graphlet_class))
    echo_results(result_df)
    plot_results(result_df,graphlet_class)
    print()